In [1]:
import numpy as np
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default='notebook'
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
import plotly.figure_factory as ff

In [2]:
freq = 200 # Hz, the accelerometer frequency
dt = 1.0/freq
dt_kf = dt*40
t_acc = np.linspace(0.0, 30.0, freq*30+1) # time vector with dt = 0.02 sec
omega = 0.1 #rad/sec
a = 10
a_truth = [a * np.sin(omega*t_acc[i]) for i in range(t_acc.shape[0])] # uncorrupted true acceleration values


In [3]:
# The process noise variance of the accelerometer with additive white Gaussian noise and zero mean:
w_var = 0.0004 #(meters/sec^2)^2
w_std = np.sqrt(w_var)
w_mean = 0
W = w_var # initial process noise covariance 

# The bias mean and std to draw from
bbar_a = 0
b_var = 0.01 # (meters/sec^2)^2
b_std = np.sqrt(b_var)

# The mean, std, and variance of the initial position to draw from
Mp_0 = 10**2 # meters^2
p_std = np.sqrt(Mp_0) # meters, the standard deviation
pbar_0 = 0

# The mean, std, and variance of the initial velocity to draw from
Mv_0 = 1 # (meters/sec)^2
v_std = np.sqrt(Mv_0) # meters/sec
vbar_0 = 100 # m/s

# measurement noise mean and variance
etabar = np.zeros((2,1))
etastd = np.array([[1, 0],[0, 0.04]]) # in units of meter^2 and (meters/sec)^2
etavar = np.dot(etastd, etastd)

H = np.array([
    [1, 0, 0],
    [0, 1, 0]
])

# initial covariance matrix
P0 = np.array([
    [Mp_0, 0, 0],
    [0, Mv_0, 0],
    [0, 0, b_var]
])

# state transition matrix
Phi = np.array([
    [1, dt_kf, -dt_kf**2/2],
    [0, 1, -dt_kf],
    [0, 0, 1]
])

Gamma = np.array([
    [-dt_kf**2/2],
    [-dt_kf],
    [0]
])

# initial position, velocity, bias guesses
vc_0 = vbar_0
pc_0 = pbar_0
ba_0 = 0





# The Kalman Filter Algorithm

In [25]:
Mk = np.zeros((3,3))
# print(Phi * Pk * np.transpose(Phi))

K_prev = np.zeros((3,2))
Mk = np.dot(Phi, np.dot(P0, np.transpose(Phi)) + np.dot(Gamma, np.dot(W, np.transpose(Gamma))))
Pk = np.linalg.inv(np.linalg.inv(Mk) + np.dot(np.transpose(H), np.dot(np.linalg.inv(etavar), H)))
Kk = np.dot(Pk, np.dot(np.transpose(H), np.linalg.inv(etavar)))
count = 1
counts = [count]
Ks = [Kk]
Ms = [Mk]
Ps = [Pk]

while count < 2 or not np.allclose(Ks[-2], Ks[-1], rtol=1.0e-09, atol=1e-09):
    Mk = np.dot(Phi, np.dot(Pk, np.transpose(Phi)) + np.dot(Gamma, np.dot(W, np.transpose(Gamma))))
    Pk = np.linalg.inv(np.linalg.inv(Mk) + np.dot(np.transpose(H), np.dot(np.linalg.inv(etavar), H)))
    Kk = np.dot(Pk, np.dot(np.transpose(H), np.linalg.inv(etavar)))
    count+=1
    counts.append(count)
    Ks.append(Kk.copy())
    Ms.append(Mk.copy())
    Ps.append(Pk.copy())
print("Converged in ", count, " iterations")
print("K")
print(Ks[-1])
print("Mk")
print(Ms[-1])
print("Pk")
print(Ps[-1])

Converged in  28790  iterations
K
[[ 7.94491235e-03  1.93210961e-01]
 [ 2.81108894e-04  9.48539758e-02]
 [-4.88381735e-08 -1.64793610e-05]]
Mk
[[ 8.06951317e-03  3.44289340e-04  9.98394914e-09]
 [ 3.13073579e-04  1.67777508e-04 -2.93555891e-08]
 [-5.43915263e-08 -2.91486581e-08  1.39038203e-08]]
Pk
[[ 7.94491235e-03  3.09137538e-04  1.55764491e-08]
 [ 2.81108894e-04  1.51766361e-04 -2.65739014e-08]
 [-4.88381735e-08 -2.63669775e-08  1.39033371e-08]]


In [23]:
print(Ks[3])
print(Ks[100])
print(Ks[-100])

[[ 2.49414648e-01  9.26512031e-02]
 [ 1.46783255e-04  5.02695227e-01]
 [-8.72435575e-05 -8.26986081e-01]]
[[ 1.19048525e-02  1.93064147e-01]
 [ 2.96418655e-04  1.05990291e-01]
 [-1.47853089e-05 -5.85428222e-03]]
[[ 7.94491216e-03  1.93210897e-01]
 [ 2.81109216e-04  9.48540845e-02]
 [-4.90068097e-08 -1.65362635e-05]]


count:  2181100
Pk
[[ 1.10524905e-01  3.33480898e-03 -7.64760480e-08]
 [ 3.33239880e-03  2.21112969e-04 -5.07511253e-09]
 [-7.64410148e-08 -5.07205193e-09  1.83626301e-10]]
Mk
[[ 1.10658338e-01  3.33923758e-03 -7.65776221e-08]
 [ 3.33682419e-03  2.21273173e-04 -5.07878738e-09]
 [-7.65425277e-08 -5.07572679e-09  1.83626385e-10]]

# The Monte Carlo Simulation

In [18]:
def monte_carlo_run():
    # Phi, Gamma, W, Etavar, P0, H are already defined
    
    # initialize the true position, velocity, and bias for this Monte Carlo run
    p_0_true = np.random.normal(pbar_0, p_std)
    v_0_true = np.random.normal(vbar_0, v_std)
    b_a_true = np.random.normal(bbar_a, b_std)
    
    # initialize the belief values for the position, velocity, bias, and state vector
    pc_now = pbar_0
    vc_now = vbar_0
    
    # variables to hold the true position and velocity
    ptrue_now = p_0_true
    vtrue_now = v_0_true
    
    # dxhat_0
    dxhat = np.array([
    [0],
    [0],
    [0]])
    
    # P0
    Pk = P0.copy()
    
    # initialize Kalman gain
#     K = np.zeros((3,2))
    K = Ks[-1]


    # record the dxhat, t, and Pk values for plotting
    dxlist = [dxhat]
    tlist = [0]
    Pklist = [Pk]
    Klist = [K]
    ptruelist = [p_0_true]
    vtruelist = [v_0_true]
    pclist = [pc_now]
    vclist = [vc_now]
    
    
    # run the MC through the time vector
    for i  in range(1, t_acc.shape[0]):
        if i % 40 == 0:
            pc_now, ptrue_now, vc_now, vtrue_now, dxhat, t = gps_timestep(i, b_a_true, pc_now, vc_now, p_0_true, v_0_true, dxhat, Pk, K)
#             pc_now = dxhat[0,0] + pc_now
#             vc_now = dxhat[1,0] + vc_now
            dxlist.append(dxhat.copy())
            tlist.append(t)
#             Pklist.append(Pk.copy())
#             Klist.append(K.copy())
            ptruelist.append(ptrue_now)
            vtruelist.append(vtrue_now)
            pclist.append(pc_now)
            vclist.append(vc_now)
        else:
            pc_now, vc_now = acc_timestep(i, b_a_true, pc_now, vc_now, dxhat[2,0])
    
    return b_a_true, dxlist, ptruelist, pclist, vtruelist, vclist, tlist

def gps_timestep(i, ba_true, pc_prev, vc_prev, p0_true, v0_true, dxhat_i, Pk_prev, K):
    # Generates the ith timestep computations of the KF update given both GPS and accelerometer updates
    # i: an int representing the time vec to index into the true acceleration vector, a_truth
    # ba_true: the true bias (a float) for this MC run to be used in the calculation of the noisy accelerometer reading
    # pc_prev: a float representing pc at timestep i-1
    # vc_prev: a float representing vc at timestep i-1
    
    ## A priori
    xbar_next = np.dot(Phi, dxhat_i)
#     Mk_next = np.dot(Phi, np.dot(Pk_prev, np.transpose(Phi)) + np.dot(Gamma, np.dot(W, np.transpose(Gamma))))
    
    ## A posteriori
    # update the Kalman gain
#     HMHT = np.dot(H, np.dot(Mk_next, np.transpose(H)))
#     K = np.dot(Mk_next,np.dot(np.transpose(H), np.linalg.inv(HMHT + etavar)))
#     print(K)
    
    # calculate the true position and velocity
    vtrue_i = v0_true + a/omega - a/omega*np.cos(omega*t_acc[i])
    ptrue_i = p0_true + (v0_true + a/omega)*t_acc[i] - a/omega**2 * np.sin(omega*t_acc[i])
    
    # generate a noisy GPS measurement for this timestep, i
    eta_i = np.random.normal(etabar, etastd) # noise on the GPS
    vgps_i = vtrue_i + eta_i[0,0]
    pgps_i = ptrue_i + eta_i[1,1]
    
    # generate an accelerometer reading and position & velocity update
    pc_i, vc_i = acc_timestep(i, ba_true, pc_prev, vc_prev, dxhat_i[2,0])
    zk = np.array([
        [pgps_i - pc_i],
        [vgps_i - vc_i]
    ])
    
    # calculate the residual
    residual = zk - np.dot(H, xbar_next)
#     print(residual)
    
    # update the state estimate, and covariance
    dxhat = xbar_next + np.dot(K, residual)
#     Pk_new = np.linalg.inv(np.linalg.inv(Mk_next) + np.dot(np.transpose(H), np.dot(np.linalg.inv(etavar), H)))
    
    return pc_i, ptrue_i, vc_i, vtrue_i, dxhat, t_acc[i]

def acc_timestep(i, ba_true, pc_prev, vc_prev, b_est):
    # Generates the ith timestep computations of position and velocity, pc and vc
    # by sampling an accelerometer reading around the true value for this Monte Carlo run
    # i: an int representing the time vec to index into the true acceleration vector, a_truth
    # ba_true: the true bias (a float) for this MC run to be used in the calculation of the noisy accelerometer reading
    # pc_prev: a float representing pc at timestep i-1
    # vc_prev: a float representing vc at timestep i-1
    
    # draw a noise value for the accelerometer
    w_t = np.random.normal(w_mean, w_std)
    
    # gather an accelerometer measurement for this timestep
    ac_prev = a_truth[i-1] + ba_true + w_t - b_est
    
    # integrate the accelerometer measurment by the euler formula
    vc_i = vc_prev + ac_prev * dt
    pc_i = pc_prev + vc_prev * dt + ac_prev * dt**2/2
    
    return pc_i, vc_i
    

In [21]:
b_true, dxlist, plist, pclist, vlist, vclist, tlist = monte_carlo_run()
# print(b_true)
# print(tlist)
y = [dxlist[i][2,0] for i in range(len(dxlist))]
dp = [dxlist[i][0,0] for i in range(len(dxlist))]
dv = [dxlist[i][1,0] for i in range(len(dxlist))]
# print(y)
fig = go.Figure()
fig.add_trace(go.Scatter(x=tlist, y=y, name="Estimated bias"))
fig.add_trace(go.Scatter(x=tlist, y=[b_true for i in range(len(dxlist))], name="True bias"))
fig.show()
fig2 = go.Figure()
fig2.add_trace(go.Scatter(x=tlist, y=dp, name="Delta Position"))
# fig2.add_trace(go.Scatter(x=tlist, y=plist, name="True Position"))
# fig2.add_trace(go.Scatter(x=tlist, y=plist, name="True Position"))
# fig2.add_trace(go.Scatter(x=tlist, y=pclist, name="Computed Position"))
fig2.show()

In [167]:
w_std

0.02

In [239]:
Ks[-1]

array([[ 1.10048246e-02,  3.33380840e-02],
       [ 3.30985730e-04,  2.20212271e-03],
       [-2.48660161e-08, -1.65439214e-07]])